# MILe

[http://export.arxiv.org/pdf/2111.12172v1](http://export.arxiv.org/pdf/2111.12172v1)

In [ ]:
using_colab = True

if using_colab:
  ! pip install -q datasets
  ! pip install -q huggingface_hub

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.datasets import ImageNet
from torchvision.models import resnet18
from PIL import Image
from datasets import load_dataset

import warnings
warnings.filterwarnings("ignore")

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Using {device}')

Using cpu


In [5]:
dataset = load_dataset('imagenet-1k')

KeyboardInterrupt: 

In [ ]:
n_epochs = 2 # equivalent of tau in the article
kt = 500 # << N
ks =  200 # <= kt
lr = 0.001
rho = 0.25
student_net = resnet18(weights='DEFAULT', progress=False)
teacher_net = resnet18(progress=False)

teacher_optimizer = SGD(teacher_net.parameters(), lr=lr, momentum=0.9)
student_optimizer = SGD(student_net.parameters(), lr=lr, momentum=0.9)
teacher_loss = nn.BCELoss()
student_loss = nn.BCELoss()

t_train_losses = []
t_val_losses = []
s_train_losses = []
s_val_losses = []

for epoch in range(n_epochs):
    t_loss = 0
    s_loss = 0
    # copy student weights to teacher net
    teacher_net.load_state_dict(student_net.state_dict())

    print('Training teacher net')
    teacher_net.train()
    for i in range(kt):
        batch_idx = np.random.randint(len(training_loader))
        x_i, y_i = training_loader[batch_idx]
        teacher_optimizer.zero_grad()
        yhat_i = teacher_net(x_i)
        loss = teacher_loss(yhat_i, y_i)
        loss.backward()
        teacher_optimizer.step()
        t_loss += loss.item()
    t_train_losses.append(t_loss)

    print('Evaluating teacher net')
    teacher_net.eval()
    with torch.no_grad():
        v_loss = 0
        for i, (v_inputs, v_labels) in enumerate(val_loader):
            v_ouputs = teacher_net(v_inputs)
            v_loss += teacher_loss(v_outputs,v_labels)
        t_val_losses.append(v_loss)

    print('Training student net')
    student_model.train()
    for i in range(ks):
        batch_idx = np.random.randint(len(training_loader))
        x_i,_ = training_loader[batch_idx]
        student_optimizer.zero_grad()

        yhat_i = (F.sigmoid(teacher_net(x_i)) > rho).int()
        ytilde_i = student_net(x_i)
        loss = student_loss(ytilde_i, yhat_i)
        loss.backward()
        student_optimizer.step()
        s_loss += loss.item()
    s_train_losses.append(s_loss)

    print('Evaluating student net')
    student_net.eval()
    with torch.no_grad():
        v_loss = 0
        for i, (v_inputs,_) in enumerate(val_loader):
            v_yhat = (F.sigmoid(teacher_net(v_inputs)) > rho).int()
            v_ytilde = student_net(v_inputs)
            v_loss += student_loss(v_yhat, v_ytilde)
        s_val_losses.append(v_loss)

<bound method Module.state_dict of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
